In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id = '1NGr179sHIt30XmQOEdJItlFRwgcnNFRh'
file = drive.CreateFile({'id':id}) 
file.GetContentFile('all_review.csv')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load in Dataset

In [ ]:
df=pd.read_csv('all_review.csv', index_col=False)
df.head()

,Name,Review,Rating,#ThumbsUp,Date&Time,App,language
0,linah sibanda,on this app i cant find a partner,5,0,18-02-2022 01:19,Tinder,en
1,Norman Johnson,tinder would be so much good if we could speci...,3,0,18-02-2022 01:16,Tinder,en
2,David Hume,still do not correctly notify match or message...,1,0,18-02-2022 01:11,Tinder,en
3,Last 1 Standing,get block because i update my bio to say `` i ...,2,0,18-02-2022 01:11,Tinder,en
4,Arthur Magamedov,love it !,5,0,18-02-2022 01:06,Tinder,en


In [ ]:
#focus on reviews that have at least one thumbup
df=df[df['#ThumbsUp']>=1].reset_index()

In [ ]:
df.shape

(110966, 8)

In [ ]:
df.App.unique()

array(['Tinder', 'Bumble', 'Hinge'], dtype=object)

In [ ]:
bumble_posi_df=df[((df.Rating==4)|(df.Rating==5))&(df.App=='Bumble')]
bumble_posi_reviews = bumble_posi_df['Review'].tolist()

In [ ]:
bumble_posi_df.shape

(3978, 8)

In [ ]:
hinge_posi_df=df[((df.Rating==4)|(df.Rating==5))&(df.App=='Hinge')]
hinge_posi_reviews = hinge_posi_df['Review'].tolist()

In [ ]:
hinge_posi_df.shape

(3426, 8)

In [ ]:
tinder_posi_df=df[((df.Rating==4)|(df.Rating==5))&(df.App=='Tinder')]
tinder_posi_reviews = tinder_posi_df['Review'].tolist()

In [ ]:
tinder_posi_df.shape

(9643, 8)

# Install packages and load in model


In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 79 kB 3.7 MB/s 
     |████████████████████████████████| 4.0 MB 32.7 MB/s 
     |████████████████████████████████| 1.2 MB 38.3 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 38.6 MB/s 
     |████████████████████████████████| 880 kB 44.1 MB/s 
     |████████████████████████████████| 6.6 MB 49.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=7dff94e68b6ca8b35c9ca87e80f93d2e1079388d92183bb53d6e2c9553a7a450
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=701f52483664ada4ce77fa7edfe1f93a3a530f8daca75f79fb4d348314c2907e
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sentence-transformers sa

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

# Embeddings

In [ ]:
embeddings = model.encode(hinge_posi_reviews, show_progress_bar=True)

Batches:   0%|          | 0/108 [00:00<?, ?it/s]

In [ ]:
embeddings.shape

(3426, 768)

# Dimensionality reduction

In [ ]:
!pip install umap-learn

     |████████████████████████████████| 88 kB 3.5 MB/s 
     |████████████████████████████████| 1.1 MB 42.2 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=b202f0120ea77d1076de62634953f71fcfa6614efe19ae2472f42ed45aa1b997
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.6-py3-none-any.whl size=53943 sha256=11d9651e22f4e1997d4acb2a0508c369d8adddee45f88f1832c0303e8a1a17d8
  Stored in directory: /root/.cache/pip/wheels/03/f1/56/f80d72741e400345b5a5b50ec3d929aca581bf45e0225d5c50
Successfully built umap-learn pynndescent


In [ ]:
import umap
umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=5, 
                            metric='cosine').fit_transform(embeddings)

# Clustering

In [ ]:
!pip install hdbscan

     |████████████████████████████████| 5.2 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2330861 sha256=a4b624b2e424363d59efaae52f06a292cd0c6e28df0a8e045a8ffb0c4e241ca8
  Stored in directory: /root/.cache/pip/wheels/6e/7a/5e/259ccc841c085fc41b99ef4a71e896b62f5161f2bc8a14c97a
Successfully built hdbscan


In [ ]:
import hdbscan
cluster = hdbscan.HDBSCAN(min_cluster_size=15,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)

# C-TF-IDF

In [ ]:
docs_df = pd.DataFrame(hinge_posi_reviews, columns=["Doc"])
docs_df['Topic'] = cluster.labels_
docs_df['Doc_ID'] = range(len(docs_df))
docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(2, 2)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count
  
tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m=len(bumble_posi_reviews))

# Topic representation

In [ ]:
def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

## Bumble Positive Reviews - BERT

In [ ]:
top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Topic,Size
0,-1,2556
4,3,356
7,6,147
2,1,142
19,18,117
1,0,74
6,5,72
15,14,69
3,2,64
20,19,50


In [ ]:
top_n_words[3][:10]

[('female make', 0.055881027814532074),
 ('love female', 0.04358339427138627),
 ('good female', 0.025604039620685203),
 ('fact female', 0.022937257196915757),
 ('female message', 0.021791697135693136),
 ('easy use', 0.021035061055218154),
 ('love fact', 0.018768025706761098),
 ('good app', 0.01816321793968123),
 ('female good', 0.017751321586190853),
 ('female love', 0.017106473624986594)]

In [ ]:
top_n_words[6][:10]

[('good app', 0.02300204681371847),
 ('easy use', 0.02008168679198962),
 ('love app', 0.01705184342219392),
 ('work good', 0.015570300243205602),
 ('24 hour', 0.013366383709309114),
 ('really love', 0.013113840002632089),
 ('app good', 0.012428646319712315),
 ('app really', 0.011902286139507933),
 ('swipe leave', 0.010978927824579598),
 ('far good', 0.009808418850254928)]

In [ ]:
top_n_words[1][:10]

[('far away', 0.029822814600135604),
 ('people far', 0.02279960679706517),
 ('just ugly', 0.02279960679706517),
 ('ugly app', 0.018593359311760028),
 ('good concept', 0.017936654326216137),
 ('country good', 0.017600788667676396),
 ('lose time', 0.017600788667676396),
 ('good location', 0.016350297544626233),
 ('make pretty', 0.016350297544626233),
 ('conversation really', 0.016350297544626233)]

In [ ]:
top_n_words[18][:10]

[('customer service', 0.014009171482060993),
 ('new message', 0.01367586204275243),
 ('wo let', 0.012798996669231216),
 ('cancel subscription', 0.011495467712475949),
 ('use app', 0.011104349350440917),
 ('facebook use', 0.009773342325436078),
 ('reinstall app', 0.009382292838641632),
 ('account delete', 0.009382292838641632),
 ('app photo', 0.009382292838641632),
 ('send new', 0.009382292838641632)]

In [ ]:
top_n_words[0][:10]

[('tinder good', 0.09855068170585989),
 ('good tinder', 0.0734197550255799),
 ('good good', 0.05620786982425519),
 ('tinder quality', 0.03905083433858083),
 ('good competitor', 0.03905083433858083),
 ('site way', 0.03905083433858083),
 ('guy tinder', 0.03696618082645659),
 ('tinder way', 0.03548709562104898),
 ('tinder easy', 0.03433982800503762),
 ('free good', 0.03433982800503762)]

In [ ]:
top_n_words[5][:10]

[('dating app', 0.020406792144179807),
 ('dating apps', 0.019199213773414766),
 ('love female', 0.015242316005020228),
 ('good dating', 0.015236929615819213),
 ('dating bumble', 0.014152795233706241),
 ('bumble dating', 0.013534111835450136),
 ('online dating', 0.012460316527960296),
 ('year love', 0.0109675425290689),
 ('wonderful female', 0.0109675425290689),
 ('meet current', 0.01061297120812471)]

In [ ]:
top_n_words[14][:10]

[('easy use', 0.43403594730901035),
 ('use easy', 0.1771208154336916),
 ('use good', 0.15796472551233734),
 ('concept easy', 0.08437370117124497),
 ('good concept', 0.05947340334626049),
 ('use fun', 0.05830470162097278),
 ('good easy', 0.04899777196161463),
 ('use quick', 0.04668780630354421),
 ('match easy', 0.04481974089801317),
 ('use love', 0.04344051684667035)]

In [ ]:
top_n_words[2][:10]

[('good dating', 0.39522484255753687),
 ('dating app', 0.3793957304747513),
 ('app good', 0.1692953471764672),
 ('app use', 0.09125722084796337),
 ('far good', 0.07125558731728607),
 ('app _emoji_', 0.06118287116101548),
 ('far far', 0.057742276846493946),
 ('use far', 0.054496563953041734),
 ('use good', 0.052146983341693355),
 ('bumble good', 0.04830784522521751)]

In [ ]:
top_n_words[9][:10]

[('allow male', 0.024139388431431416),
 ('hate hate', 0.020523088974536082),
 ('female interested', 0.01738718329803339),
 ('love female', 0.017113161642173046),
 ('good tinder', 0.016983994234628083),
 ('attractive female', 0.015744849868242063),
 ('good app', 0.01521460619223536),
 ('female match', 0.014708963587492203),
 ('male female', 0.014708963587492203),
 ('make feel', 0.01458748038743855)]

## Hinge Positive Reviews - BERT

In [ ]:
top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Topic,Size
0,-1,2165
3,2,146
10,9,131
7,6,106
18,17,98
6,5,98
13,12,90
12,11,86
20,19,73
11,10,62


In [ ]:
top_n_words[2][:10]

[('good tinder', 0.29265548395968893),
 ('tinder bumble', 0.194888622127947),
 ('bumble good', 0.11637830036439599),
 ('tinder good', 0.10621803233488314),
 ('good bumble', 0.0877744972779424),
 ('way good', 0.0722505609732934),
 ('bumble tinder', 0.05368956831341607),
 ('good good', 0.03780993124061471),
 ('good okcupid', 0.03232226363276548),
 ('effort good', 0.025191997608139213)]

In [ ]:
top_n_words[9][:10]

[('good far', 0.09502602913776788),
 ('good concept', 0.08887564812324415),
 ('good good', 0.08611667058768901),
 ('quality match', 0.07950623025670867),
 ('far good', 0.07296612480892206),
 ('match good', 0.05576474367623165),
 ('need good', 0.05521338518571555),
 ('concept good', 0.04687772581167822),
 ('good experience', 0.045779481216595565),
 ('high quality', 0.041823557757173745)]

In [ ]:
top_n_words[6][:10]

[('love app', 0.026092180335367733),
 ('good app', 0.020663641677473844),
 ('make easy', 0.01772892371406273),
 ('good love', 0.017127685267403058),
 ('far good', 0.016768907589550446),
 ('easy use', 0.015626392857913448),
 ('love format', 0.015463239236827188),
 ('use love', 0.014341232492010166),
 ('app really', 0.013954856934647542),
 ('easy start', 0.013695562493994513)]

In [ ]:
top_n_words[17][:10]

[('dating apps', 0.02042522104983578),
 ('use app', 0.011567650913317214),
 ('app month', 0.009938524078005734),
 ('apps use', 0.009916626549097066),
 ('use hinge', 0.009428794216271385),
 ('delete app', 0.00904278051988581),
 ('far good', 0.009018572149001921),
 ('hinge good', 0.008639941068150851),
 ('good step', 0.008631359148554391),
 ('meet boyfriend', 0.008631359148554391)]

In [ ]:
top_n_words[5][:10]

[('good app', 0.0230096368485895),
 ('start conversation', 0.012877193835253589),
 ('good people', 0.01221323568591531),
 ('push notification', 0.01035261719967583),
 ('overall good', 0.010104690257688887),
 ('pretty good', 0.009901410280124767),
 ('good conversation', 0.00965886239302129),
 ('work good', 0.009365515748415572),
 ('good experience', 0.008875305837795604),
 ('little glitchy', 0.008803730070711535)]

In [ ]:
top_n_words[12][:10]

[('good dating', 0.24860249108368154),
 ('dating app', 0.24632053764401177),
 ('dating good', 0.08243376214950043),
 ('app use', 0.0612617758573942),
 ('love good', 0.05779069526225208),
 ('app good', 0.05512640720798744),
 ('far good', 0.053548943986589374),
 ('app experience', 0.05163486833874496),
 ('dating site', 0.04528803354147185),
 ('app far', 0.04260789727014025)]

In [ ]:
top_n_words[11][:10]

[('easy use', 0.06667260952709739),
 ('way meet', 0.06166031060580065),
 ('conversation starter', 0.048842809587669296),
 ('meet people', 0.04596791763737024),
 ('good conversation', 0.04509450421849529),
 ('good people', 0.040728680470539555),
 ('interesting people', 0.03730134095971078),
 ('good way', 0.03707086103275633),
 ('use love', 0.03388943555035018),
 ('really love', 0.03287079660618463)]

In [ ]:
top_n_words[19][:10]

[('dating app', 0.0367903258709079),
 ('dating apps', 0.01855420843458365),
 ('good dating', 0.017553992691023312),
 ('app use', 0.011246913811987254),
 ('online dating', 0.010987894518732305),
 ('scam profile', 0.010715005128519589),
 ('bad review', 0.010537771142433629),
 ('just love', 0.010403014003187728),
 ('location good', 0.010197093964854683),
 ('love dating', 0.010163252098543692)]

In [ ]:
top_n_words[10][:10]

[('easy use', 0.2678810204213734),
 ('use good', 0.10795606364654489),
 ('people easy', 0.0841519883802221),
 ('use easy', 0.07879922481677337),
 ('good easy', 0.07300686612951388),
 ('good clean', 0.0715617487176409),
 ('easy navigate', 0.062181726531706255),
 ('lot people', 0.05618584308954101),
 ('intuitive easy', 0.054252766277528376),
 ('user friendly', 0.052013006302808265)]

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
np.savetxt("hinge_posi_embeddings.csv", embeddings, delimiter=",")
!cp hinge_posi_embeddings.csv "drive/My Drive/"

In [ ]:
id = '1-1-5cC_Az4eQIZ-D4zrob0m81X1Tof76'
file = drive.CreateFile({'id':id}) 
file.GetContentFile('hinge_nega_embeddings.csv')

In [ ]:
test=pd.read_csv('hinge_nega_embeddings.csv', header=None)
embeddings=test.to_numpy()

In [ ]:
embeddings.shape

(3426, 768)

## Reference:
[Topic Modeling with BERT](https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6)